## Preliminary Functions

In [2]:
# Function to encode a string into an integer
def encode_message(m):
    # Convert each character to an 8-bit binary string
    bin_str = ''.join(format(ord(c), '08b') for c in m)
    
    # Convert the binary string into an integer
    z = int(bin_str, 2)
    return z


# Function to decode integers into a string
def decode_message(z):
    # Convert the integer to a binary string
    bin_str = bin(z)[2:]  # Strip the '0b' prefix
    
    # Ensure the binary string length is a multiple of 8
    padding = len(bin_str) % 8
    if padding != 0:
        bin_str = '0' * (8 - padding) + bin_str
    
    # Split the binary string into 8-bit chunks and convert them to characters
    decoded_message = ''.join([chr(int(bin_str[i:i + 8], 2)) for i in range(0, len(bin_str), 8)])
    
    return decoded_message

# Function to find the modular inverse of a number a modulo m
def Invert(a, m):
    x, y, gcd = extended_gcd(a, m)
    if gcd != 1:
        return None  # check for no modular inverses
    return x % m 

# Extended Euclidean Algorithm to find the greatest common divisor (GCD) and coefficients
def extended_gcd(a, b):
    if b == 0:
        return 1, 0, a # base case
    
    x1, y1, gcd = extended_gcd(b, a % b) # induction

    x, y = y1, x1 - (a // b) * y1 
    return x, y, gcd


In [3]:
# Function for generating primes
import sympy

def generate_prime(bit_length):
    """Generates a prime number with the specified bit length."""
     
    # Generate a random prime of the required bit length
    prime = sympy.randprime(2**(bit_length - 1), 2**bit_length)

    return prime

## ElGamal

In [4]:
import secrets

class ElGamal:
    def __init__(self, p, g):
        """
        Initialise the ElGamal with the prime 'p' and generator 'g'.
        """

        self.p = p # Prime modulus
        self.g = g # Primitive root for mod p

        self.private_key = 0 # Private key 
        self.public_key = 0 # Public key 

        self.c1 = 0 # Cipher text component 
        self.c2 = 0 # Cipher text component

    def key(self, private_key):
        """
        Generate the public key from the private key
        """

        self.private = private_key 

        # check private key is valid 
        if (private_key >= 1) and (self.p - 1 >= private_key):
            self.public_key = pow(self.g, self.private_key, self.p) 
        else:
            raise ValueError("Invalid private key.")

        # return public key to be published
        return self.public_key

    def encrypt(self, message, public_key):
        """
        Encrypts a message using the ElGamal encryption scheme.
        """

        # Generate random ephemeral key k
        self.k = secrets.randbelow(self.p)

        # Encode message as an integer
        self.z = encode_message(message)

        # Create ciphertext
        self.public_key = public_key
            
        self.c1 = pow(self.g, self.k, self.p)
        self.c2 = self.z * pow(self.public_key, self.k) % self.p 

        # return ciphertexts
        return [self.c1, self.c2]

    
    def decrypt(self,c1,c2):
        """
        Decrypts the ciphertext (c1, c2) using the private key.
        """
        
        self.c1 = c1 
        self.c2 = c2

        # Calculate decrypted integer
        self.decrypt_int =  (Invert(pow(self.c1,self.private_key), self.p) * c2) % self.p

        # Decode decrypted integer as a string
        self.msg = decode_message(self.decrypt_int)

        return self.msg


In [5]:
## Example:

# Alice creates a public key using a private key
p = generate_prime(1080)

Alice = ElGamal(p,2) # publicly chosen prime and base
A = Alice.key(67) # choose a prive key (67) and generate public key A

# Bob encrypts a message using Alice's public key
Bob = ElGamal(p,2)

message = "Let me come in where you are weeping, friend, And let me take your hand. I, who have known a sorrow such as yours, Can understand... "

c1,c2 = Bob.encrypt(message, A) # Encrypt 'b' with Alice's public key A

# Alice decrypts Bob's cipher text
Alice.decrypt(c1,c2)


'Let me come in where you are weeping, friend, And let me take your hand. I, who have known a sorrow such as yours, Can understand... '

## RSA

In [10]:
class RSA:
    def __init__(self, e):
        """
        Initialise class with exponent e. 
        """
        self.e = e
    
    def key(self, p, q):
        """
        Calculate public key.
        """

        self.p = p # private key p
        self.q = q # private key q
        self.N = self.p*self.q # public key

        # Check encryption conponent is compntaible with secret keys
        x, y, gcd = extended_gcd(self.e,(p - 1)*(q - 1))
        if gcd == 1:
            return [self.N, self.e]
        else:
            raise ValueError("Private Keys incompatible with encryption exponent.")


    def encrypt(self, public_key, plain_text):
        """
        Encrypt plain text and return ciphertext. 
        """

        self.N, self.e = public_key

        # Encode plaintext as an integer 
        self.z = encode_message(plain_text)

        # Check plain text can be encoded using public keys
        if self.z >= 1 and self.z < self.N:
            self.c = pow(self.z, public_key[1], self.N)
        else:
            print("Plain text too long to encrypt with given public key.")

        return self.c
    
    def decrypt(self, c):
        """ 
        Decrypt cipher text
        """

        # Calculate inverse of the exponent
        self.d = Invert(self.e, (self.p - 1)*(self.q - 1))

        # Recover interger 
        self.msg = pow(c, self.d, self.N)

        # Convert integer to string
        self.msg = decode_message(self.msg)

        return self.msg
        

In [11]:
## Example:

# Alice chooses two primes as a private key with a compatible exponent
p = generate_prime(1080)
q = generate_prime(1080)
e = 65537

# Alice generates sa public key
Alice = RSA(e) 
A = Alice.key(p,q) 

# Bob encrypts a message using Alice's public key
Bob = RSA(e)
message = "Let me come in - I would be very still. Beside you in your grief; I would not bid you cease your weeping, friend, Tears can bring relief."

c = Bob.encrypt(A, message) # Encrypt 'b' with Alice's public key A

# Alice decrypts Bob's cipher text
Alice.decrypt(c)


'Let me come in - I would be very still. Beside you in your grief; I would not bid you cease your weeping, friend, Tears can bring relief.'